# I: The Oracle (Using Quantum Phase Estimation)

This oracle marks the target state |x> by flipping its phase.
It works in three steps:
## 1. QPE: Estimate the phase of each |x> in a superposition.
We initialize the register with n data qubits and d ancillas. Then we apply the Hadamard gate to get an equal superposition of all n qubit states. After we create the Grover oracle by using Quantum Phase Estimation (QPE). This is done by applying Hadamards on the d ancillas and then controlled U**(2j) operations on all the n qubit register, which is controlled by the jth ancilla. We finish this step by doing a QFT$^\dagger$. 
## 2. Mark: Flip the phase of the state whose estimated phase matches the target.
Now that the phase is stored in the ancilla register, we can mark the one we're looking for.
Target Phase: The input $t\in [0,1]$ gives us the target phase. Since $2^d\theta(x)$ is an integer, the target integer we are looking for in the ancilla register is $T=2^dt$. We then need to flip the sign if and only if the ancilla register is in the state $∣T\rangle$. This is done with a multi-controlled-Z (MCZ) or multi-controlled-Phase gate.
## 3. Un-QPE: Reverse the QPE step to clean up the ancilla qubits.
We apply the inverse circuit of step 1 to take the ancillas back to the original state

# II: THE Diffuser (Amplitude Amplification)
We use the standard Grover diffuser operator. It amplifies the amplitude of the marked state (the one with the negative phase).

In [5]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import QFT, Diagonal, MCPhaseGate
from qiskit.quantum_info import Statevector, Operator
import math


def create_qpe_oracle(n: int, d: int, U_diagonals: list, t: float) -> QuantumCircuit:
    
    # Define the quantum registers
    main_qubits = QuantumRegister(n, name='main')
    ancilla_qubits = QuantumRegister(d, name='ancilla')
    oracle_circuit = QuantumCircuit(main_qubits, ancilla_qubits, name='Oracle')

    # --- 1. QPE Sub-circuit ---
    # Create the circuit that performs the phase estimation
    qpe_subcircuit = QuantumCircuit(main_qubits, ancilla_qubits, name='QPE')

    # Apply Hadamards to ancilla qubits
    qpe_subcircuit.h(ancilla_qubits)

    # Apply controlled-U^{2^j} gates
    for j in range(d):
        power_of_2 = 2**j
        # Classically compute the diagonals for U^{2^j} 
        U_power_diagonals = [val**power_of_2 for val in U_diagonals]
        # Create the U^{2^j} gate 
        U_power_gate = Diagonal(U_power_diagonals)
        # Create the controlled version of the gate 
        controlled_U_gate = U_power_gate.control(1, ctrl_state='1')
        
        # Apply the gate
        qpe_subcircuit.append(controlled_U_gate, [ancilla_qubits[j]] + main_qubits[:])

    # Apply the inverse QFT on the ancilla register 
    qpe_subcircuit.append(QFT(num_qubits=d, inverse=True), ancilla_qubits)


    # --- 2. Marking Sub-circuit ---
    # Mark the state where the ancilla register matches the target phase integer
    marking_subcircuit = QuantumCircuit(ancilla_qubits, name='Mark')
    
    # Calculate the integer representation of the target phase t 
    # The promise is that 2^d * t is an integer
    target_integer = int(round(t * (2**d)))
    
    # Convert the integer to a binary string of length d
    target_binary = format(target_integer, f'0{d}b')

    # Apply X gates to ancillas corresponding to '0's in the binary string
    for i in range(d):
        if target_binary[i] == '0':
            marking_subcircuit.x(ancilla_qubits[i])

    # Apply the multi-controlled phase gate 
    marking_subcircuit.append(MCPhaseGate(num_ctrl_qubits=d-1, lam=np.pi), ancilla_qubits)

    # Un-apply the X gates
    for i in range(d):
        if target_binary[i] == '0':
            marking_subcircuit.x(ancilla_qubits[i])


    # --- 3. Assemble the Full Oracle ---
    # Append QPE
    oracle_circuit.append(qpe_subcircuit, oracle_circuit.qubits)
    # Append Marking
    oracle_circuit.append(marking_subcircuit, ancilla_qubits)
    # Append inverse QPE to uncompute and clean up ancillas
    oracle_circuit.append(qpe_subcircuit.inverse(), oracle_circuit.qubits)
    
    return oracle_circuit


#  THE DIFFUSER (AMPLITUDE AMPLIFICATION)

def create_diffuser(n: int) -> QuantumCircuit:
  
    diffuser_circuit = QuantumCircuit(n, name='Diffuser')
    
    # Apply H-gates
    diffuser_circuit.h(range(n))
    # Apply X-gates
    diffuser_circuit.x(range(n))
    # Apply multi-controlled Z gate
    diffuser_circuit.h(n-1)
    diffuser_circuit.mcx(list(range(n-1)), n-1)
    diffuser_circuit.h(n-1)
    # Apply X-gates
    diffuser_circuit.x(range(n))
    # Apply H-gates
    diffuser_circuit.h(range(n))
    
    return diffuser_circuit


# This function brings everything together to build the final circuit

def prepare_eigenvector(U_diagonals: list, n: int, d: int, t: float) -> QuantumCircuit:
 
    # Define quantum registers
    main_qubits = QuantumRegister(n, name='main')
    ancilla_qubits = QuantumRegister(d, name='ancilla')
    circuit = QuantumCircuit(main_qubits, ancilla_qubits)

    # 1. Initialization: Create superposition over all |x>
    circuit.h(main_qubits)
    
    # 2. Determine number of Grover iterations
    N = 2**n
    num_iterations = math.floor(math.pi / 4 * math.sqrt(N))
    
    # Get the oracle and diffuser circuits
    oracle = create_qpe_oracle(n, d, U_diagonals, t)
    diffuser = create_diffuser(n)
    
    # 3. Grover Loop: Repeatedly apply oracle and diffuser
    for _ in range(num_iterations):
        # Apply Oracle 
        circuit.append(oracle, circuit.qubits)
        # Apply Diffuser 
        circuit.append(diffuser, main_qubits)
        
    return circuit


# Example and Demonstration

Here we test the function with an example for n=3, d=3.
We want to find the eigenvector |5> = |101>.

In [19]:
#Choose all the input parameters 

n_main = 3
d_ancilla = 3
    
# The total number of states is 2^n
N_states = 2**n_main
    
# 1. Define the diagonal unitary U 
# U|x> = e^{2*pi*i*(x / 2^d)}|x>
# This satisfies the promise that 2^d * theta(x) is an integer. 
phases = [x / (2**d_ancilla) for x in range(N_states)]
diagonals_U = [np.exp(2j * np.pi * p) for p in phases]
    
# 2. Define the target phase value t 
# We want to find the eigenvector |x> where x = 5.
# The corresponding phase is theta(5) = 5 / 2^3 = 5/8.
target_x = 5
target_t = phases[target_x] # t = 5/8
    
print("--- Eigenvector Preparation Demo ---")
print(f"Number of main qubits (n): {n_main}")
print(f"Number of ancilla qubits (d): {d_ancilla}")
print(f"Target eigenvector to find: |{target_x}> = |{format(target_x, f'0{n_main}b')}>")
print(f"Corresponding target phase (t): {target_t}")
print("-" * 35)
    
# Get the final quantum circuit 
# No classical bits or measurements are used in the construction. 
eigenvector_circuit = prepare_eigenvector(diagonals_U, n_main, d_ancilla, target_t)
print("\nDrawing the full circuit is very large. Drawing components instead.")
print("Oracle Circuit:")
print(create_qpe_oracle(n_main, d_ancilla, diagonals_U, target_t).draw(output="mpl", style="bw"))
print("\nDiffuser Circuit:")
print(create_diffuser(n_main).draw())

# --- Verification ---
# Since we cannot use measurement, we verify the output using a statevector simulation.
# This shows that the circuit correctly prepares the desired state.
print("Simulating the final statevector...")
final_statevector = Statevector(eigenvector_circuit)
#print(np.round(final_statevector))

probs = np.abs(final_statevector.data)**2
most_likely_state_index = np.argmax(probs)

print(f"\nResult of simulation:")
print(f"The most probable state is: |{most_likely_state_index}>")
print(f"Probability of measuring this state: {probs[most_likely_state_index]:.4f}")

# Optional: Display all probabilities
print("\nFull state probabilities:")
for i, p in enumerate(probs):
     if p > 0.001:
         print(f"  Prob(|{format(i, f'0{n_main}b')}>): {p:.4f}")

--- Eigenvector Preparation Demo ---
Number of main qubits (n): 3
Number of ancilla qubits (d): 3
Target eigenvector to find: |5> = |101>
Corresponding target phase (t): 0.625
-----------------------------------

Drawing the full circuit is very large. Drawing components instead.
Oracle Circuit:
Figure(614.444x535.111)

Diffuser Circuit:
     ┌───┐┌───┐          ┌───┐┌───┐     
q_0: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
     ├───┤├───┤       │  ├───┤├───┤     
q_1: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
     ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐
q_2: ┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘
Simulating the final statevector...

Result of simulation:
The most probable state is: |5>
Probability of measuring this state: 0.9453

Full state probabilities:
  Prob(|000>): 0.0078
  Prob(|001>): 0.0078
  Prob(|010>): 0.0078
  Prob(|011>): 0.0078
  Prob(|100>): 0.0078
  Prob(|101>): 0.9453
  Prob(|110>): 0.0078
  Prob(|111>): 0.0078
